Goals for Lab 6

1. We want to extract 30% of the lakes to be used as a validation set, with the remaining lakes separated into a train/test split (again use 70%/30% split).  In summary we should have a random split with
- 30% of the lakes marked as "Validation",
- 50% of the lakes (~70% of 70%) marked as "Training", and
- 20% of the lakes (~30% of 70%) marked as "Test".
2. For each target,
- Perform a grid search for both CART and Random Forests using the training lakes,
- Compare the best model of each type on the test lakes to determine the best overall model.
- Refit the best model on the 70% of the lakes not in the validation set (training + test).
- Use the validation set to estimate the performance of this model.
3. Write up a summary of what we learn.

In [1]:
# Display the first 10 rows of the dataset (no path variable saved)
import polars as pl

# Lazily scan and collect the first 10 rows to display
final_lazy = pl.scan_csv('data/water_quality_and_parcel_summaries_2004_to_2015.csv', separator=',', has_header=True, ignore_errors=True)
final_lazy.limit(10).collect()

,Monit_MAP_CODE1,Year,LAKE_NAME,Mean_Secchi_Depth_Result,Mean_Phosporus_Result,Mean_EMV_Total,STD_EMV_Total,Mean_Sale_Value,STD_Sale_Value,Mean_Total_Tax,STD_Total_Tax,Mean_Garage_Size,STD_Garage_Size,Mean_Fin_SQ_FT,STD_Fin_SQ_FT,Percentage_Yes_Basement,Percentage_Yes_Garage,Percentage_Yes_Tax_Exempt,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,Percentage_Air_Heating,Percentage_Space_Heating,Percentage_Water_Heating,Percentage_Electric_Heating,Percentage_Other_Heating,Percentage_No_Heating
i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""02000500-01""",2014,"""George Watch Lake""",0.716667,0.108778,215637.592745,459023.922281,122485.615829,183414.899771,3313.685903,17837.814516,null,null,1849.473207,8220.487298,70.486397,0.0,7.914262,0.0,0.0,0.0,2.308326,97.691674,0.0,0.0,0.0,0.0,85.325639,14.674361
1,"""02000500-01""",2013,"""George Watch Lake""",0.365,0.3105,196764.633141,454192.377508,120601.723001,190129.461399,3319.081616,18199.952917,null,null,1841.112119,8221.467234,69.991756,0.0,7.914262,0.824402,0.0,0.0,1.154163,0.0,79.060181,0.824402,1.154163,2.802968,1.154163,0.0
2,"""02000500-01""",2012,"""George Watch Lake""",0.359,0.2649,200414.333057,480355.968668,118315.128418,190091.029827,3460.064623,19346.923479,null,null,1811.576636,8223.512958,70.173985,0.0,8.119304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"""02000500-01""",2011,"""George Watch Lake""",0.973333,0.119417,216297.932175,536941.029067,111218.812242,190312.451914,3459.313482,19320.28833,null,null,1780.282051,8215.092878,76.840364,0.0,8.271299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"""02000500-01""",2010,"""George Watch Lake""",0.493333,0.173,222789.430223,559656.083477,110765.119736,190448.272924,3384.645747,18646.626107,null,null,1760.876135,8210.519477,75.887696,0.0,8.092486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"""02000500-01""",2009,"""George Watch Lake""",0.538,0.1056,242463.996697,591927.093142,107504.150289,190962.494434,3489.229562,19418.661942,null,null,1737.592898,8212.984261,74.566474,0.0,7.927333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"""02000500-01""",2008,"""George Watch Lake""",0.55,0.148833,251564.905037,639093.204004,98506.346821,180956.570533,3317.029727,19578.77809,null,null,1730.542527,8224.659654,73.575557,0.0,7.927333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"""02000500-01""",2007,"""George Watch Lake""",0.562857,0.203714,238607.467283,625283.21116,96000.966898,213258.70521,3230.503464,18438.443965,null,null,1675.535027,7962.522155,72.440339,0.0,8.314088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"""02000500-01""",2006,"""George Watch Lake""",0.728571,0.164286,235092.781833,609406.343953,78391.635036,174394.73552,3435.78751,23647.017611,null,null,1652.141119,8163.855783,70.072993,0.0,8.434712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Add Training, Test, and Validation column

In [2]:
# Lazily add a deterministic `split` column and show the top 10 rows (no full collect)
import polars as pl

# Compute a stable hash of each row (deterministic) and derive splits from the hash
# Map hash % 100 -> 0-29 Validation (30%), 30-79 Training (50%), 80-99 Test (20%)
final_with_split_lazy = (
    final_lazy
    .with_columns([
        pl.struct(pl.all()).hash().alias('_row_hash')
    ])
    .with_columns([
        pl.when((pl.col('_row_hash') % 100) < 30).then(pl.lit('Validation'))
          .when((pl.col('_row_hash') % 100) < 80).then(pl.lit('Training'))
          .otherwise(pl.lit('Test')).alias('split')
    ])
    .drop('_row_hash')
)

# Only collect and display the first 10 rows (lazy until this point)
final_with_split_lazy.limit(10).collect()

,Monit_MAP_CODE1,Year,LAKE_NAME,Mean_Secchi_Depth_Result,Mean_Phosporus_Result,Mean_EMV_Total,STD_EMV_Total,Mean_Sale_Value,STD_Sale_Value,Mean_Total_Tax,STD_Total_Tax,Mean_Garage_Size,STD_Garage_Size,Mean_Fin_SQ_FT,STD_Fin_SQ_FT,Percentage_Yes_Basement,Percentage_Yes_Garage,Percentage_Yes_Tax_Exempt,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,Percentage_Air_Heating,Percentage_Space_Heating,Percentage_Water_Heating,Percentage_Electric_Heating,Percentage_Other_Heating,Percentage_No_Heating,split
i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
0,"""02000500-01""",2014,"""George Watch Lake""",0.716667,0.108778,215637.592745,459023.922281,122485.615829,183414.899771,3313.685903,17837.814516,null,null,1849.473207,8220.487298,70.486397,0.0,7.914262,0.0,0.0,0.0,2.308326,97.691674,0.0,0.0,0.0,0.0,85.325639,14.674361,"""Training"""
1,"""02000500-01""",2013,"""George Watch Lake""",0.365,0.3105,196764.633141,454192.377508,120601.723001,190129.461399,3319.081616,18199.952917,null,null,1841.112119,8221.467234,69.991756,0.0,7.914262,0.824402,0.0,0.0,1.154163,0.0,79.060181,0.824402,1.154163,2.802968,1.154163,0.0,"""Validation"""
2,"""02000500-01""",2012,"""George Watch Lake""",0.359,0.2649,200414.333057,480355.968668,118315.128418,190091.029827,3460.064623,19346.923479,null,null,1811.576636,8223.512958,70.173985,0.0,8.119304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Test"""
3,"""02000500-01""",2011,"""George Watch Lake""",0.973333,0.119417,216297.932175,536941.029067,111218.812242,190312.451914,3459.313482,19320.28833,null,null,1780.282051,8215.092878,76.840364,0.0,8.271299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Test"""
4,"""02000500-01""",2010,"""George Watch Lake""",0.493333,0.173,222789.430223,559656.083477,110765.119736,190448.272924,3384.645747,18646.626107,null,null,1760.876135,8210.519477,75.887696,0.0,8.092486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Training"""
5,"""02000500-01""",2009,"""George Watch Lake""",0.538,0.1056,242463.996697,591927.093142,107504.150289,190962.494434,3489.229562,19418.661942,null,null,1737.592898,8212.984261,74.566474,0.0,7.927333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Test"""
6,"""02000500-01""",2008,"""George Watch Lake""",0.55,0.148833,251564.905037,639093.204004,98506.346821,180956.570533,3317.029727,19578.77809,null,null,1730.542527,8224.659654,73.575557,0.0,7.927333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Validation"""
7,"""02000500-01""",2007,"""George Watch Lake""",0.562857,0.203714,238607.467283,625283.21116,96000.966898,213258.70521,3230.503464,18438.443965,null,null,1675.535027,7962.522155,72.440339,0.0,8.314088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Training"""
8,"""02000500-01""",2006,"""George Watch Lake""",0.728571,0.164286,235092.781833,609406.343953,78391.635036,174394.73552,3435.78751,23647.017611,null,null,1652.141119,8163.855783,70.072993,0.0,8.434712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Training"""


## Preparing Data for Grid Search

In [3]:
final_train = final_with_split_lazy.filter(pl.col("split").is_in(["Training", "Test"]))
final_valid = final_with_split_lazy.filter(pl.col('split') == 'Validation')

In [5]:
import polars as pl
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Collect training partition to pandas
final_train_pd = final_train.collect().to_pandas()

X = final_train_pd.drop(columns=['Monit_MAP_CODE1','Year','LAKE_NAME','Mean_Secchi_Depth_Result','Mean_Phosporus_Result', 'split'])  # Would have Lake Data columns here
y = final_train_pd[['Mean_Secchi_Depth_Result','Mean_Phosporus_Result']]  # Would have Lake Data column here

# Split into train/test (30% test of the Training partition)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Cart

In [6]:
cart_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("reg", DecisionTreeRegressor())
])
cart_params = {
    "reg__max_depth": [3, 5, 10, None],
    "reg__min_samples_split": [2, 5, 10]
}
cart_grid = GridSearchCV(cart_pipe, cart_params, cv=5, scoring="r2")
cart_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('reg', DecisionTreeRegressor())]),
             param_grid={'reg__max_depth': [3, 5, 10, None],
                         'reg__min_samples_split': [2, 5, 10]},
             scoring='r2')

## RandomForestRegressor

In [7]:
rf_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("reg", RandomForestRegressor())
])
rf_params = {
    "reg__n_estimators": [50, 100],
    "reg__max_depth": [5, 10, None],
    "reg__min_samples_split": [2, 5]
}
rf_grid = GridSearchCV(rf_pipe, rf_params, cv=5, scoring="r2")
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('reg', RandomForestRegressor())]),
             param_grid={'reg__max_depth': [5, 10, None],
                         'reg__min_samples_split': [2, 5],
                         'reg__n_estimators': [50, 100]},
             scoring='r2')

In [8]:
# Convert validation set to pandas
final_valid_pd = final_valid.collect().to_pandas()

# Prepare validation features and target
X_val = final_valid_pd.drop(columns=['Monit_MAP_CODE1','Year','LAKE_NAME','Mean_Secchi_Depth_Result','Mean_Phosporus_Result', 'split'])  # Would have Lake Data column here
y_val = final_valid_pd[['Mean_Secchi_Depth_Result','Mean_Phosporus_Result']]  # Would have Lake Data column here

In [9]:
cart_preds = cart_grid.best_estimator_.predict(X_val)
rf_preds = rf_grid.best_estimator_.predict(X_val)

cart_r2 = r2_score(y_val, cart_preds)
rf_r2 = r2_score(y_val, rf_preds)

cart_rmse = np.sqrt(mean_squared_error(y_val, cart_preds))
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_preds))

print(f"CART Validation R²: {cart_r2:.3f}, RMSE: {cart_rmse:.2f}")
print(f"Random Forest Validation R²: {rf_r2:.3f}, RMSE: {rf_rmse:.2f}")

CART Validation R²: 0.351, RMSE: 0.63
Random Forest Validation R²: 0.537, RMSE: 0.49


From this medical data I was able to find that Random Forest Validation was the better method at predicting water quality.

## Here are the new questions we want to find:
- How does property value influence water quality?


In [11]:
final_train_pd = final_train.collect().to_pandas()

X = final_train_pd[['Mean_EMV_Total','STD_EMV_Total','Mean_Sale_Value','STD_Sale_Value']]  # Would have Lake Data columns here
y = final_train_pd[['Mean_Secchi_Depth_Result','Mean_Phosporus_Result']]  # Would have Lake Data column here

# Split into train/test (30% test of the Training partition)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



rf_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("reg", RandomForestRegressor())
])
rf_params = {
    "reg__n_estimators": [50, 100],
    "reg__max_depth": [5, 10, None],
    "reg__min_samples_split": [2, 5]
}
rf_grid = GridSearchCV(rf_pipe, rf_params, cv=5, scoring="r2")
rf_grid.fit(X_train, y_train)



# Convert validation set to pandas
final_valid_pd = final_valid.collect().to_pandas()

# Prepare validation features and target
X_val = final_valid_pd[['Mean_EMV_Total','STD_EMV_Total','Mean_Sale_Value','STD_Sale_Value']]  # Would have Lake Data column here
y_val = final_valid_pd[['Mean_Secchi_Depth_Result','Mean_Phosporus_Result']]  # Would have Lake Data column here



rf_preds = rf_grid.best_estimator_.predict(X_val)

rf_r2 = r2_score(y_val, rf_preds)

rf_rmse = np.sqrt(mean_squared_error(y_val, rf_preds))

print(f"Random Forest Validation R²: {rf_r2:.3f}, RMSE: {rf_rmse:.2f}")

Random Forest Validation R²: 0.423, RMSE: 0.58


## Summary
From this output I can see that property value variables seem to predict a high percentage of the variablity of water quality from all of the variables I have in my data. I came to this conclusion since from my initial Random Forest Validation that included all variables had an R^2 value of 0.537 and rerunning the Random Forest Validation with just the porperty value variables returned an R^2 value of 0.423.